In [1]:
## Import all packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, NavigableString
from urllib.request import urlopen
import os, re
import numpy as np
import pubchempy as pcp
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from mordred import Calculator, descriptors
trainingSetDF = pd.read_excel("DrugInformation_ImmuneModPaper.xlsx")
###Drop duplicates (certain drugs listed multiple times with slightly different smiles)
trainingSetDF = trainingSetDF.drop_duplicates(subset = "CHEMBL_ID")
##Remove compounds without ChEMBL IDs
trainingSetDF = trainingSetDF[~pd.isna(trainingSetDF.CHEMBL_ID)]
##Using sturcture links_1 because chembl_ID was written differently.
drugBankDF = pd.read_csv("structure links_1.csv")
##Remove compounds without ChEMBL IDs
drugBankDF = drugBankDF[~pd.isna(drugBankDF.CHEMBL_ID)]
##How many drugs overlap between the training set and drugbank?
overlap = len(set(trainingSetDF.CHEMBL_ID.unique()).intersection(set(drugBankDF.CHEMBL_ID.unique())))
print(overlap)
##Merge
combinedDF = pd.merge(trainingSetDF, drugBankDF, on= "CHEMBL_ID", how = "inner")
assert combinedDF.shape[0] == overlap

747


In [3]:
## molecular calculator
calc = Calculator(descriptors, ignore_3D = True)

In [4]:
# use SMILES to calculate 
mols = [Chem.MolFromSmiles(smi) for smi in combinedDF['SMILES_x']]
## Create a dataframe after calculation
calDF = calc.pandas(mols)

C:\Users\eric\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


C:\Users\eric\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


C:\Users\eric\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


C:\Users\eric\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|████████████████████████████████████████████████████████████████████████████████| 747/747 [00:42<00:00, 17.69it/s]


In [5]:
## Adding CHEMBL_ID as index
calDF = calDF.set_index(x for x in combinedDF['CHEMBL_ID'])

KeyError: "None of [Index(['CHEMBL222280', 'CHEMBL844', 'CHEMBL388590', 'CHEMBL1201287',\n       'CHEMBL1199080', 'CHEMBL601719', 'CHEMBL1293', 'CHEMBL841', 'CHEMBL954',\n       'CHEMBL13960',\n       ...\n       'CHEMBL625', 'CHEMBL1201224', 'CHEMBL927', 'CHEMBL902', 'CHEMBL1027',\n       'CHEMBL996', 'CHEMBL1175', 'CHEMBL163', 'CHEMBL1472975', 'CHEMBL499'],\n      dtype='object', length=747)] are in the [columns]"

In [8]:
print(len(combinedDF['CHEMBL_ID']))

747


In [7]:
calDF

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.003721,11.805899,0,0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505,26,88.0,104.0,7.027778,3.777778
1,13.423098,11.209659,0,3,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515,24,90.0,106.0,4.305556,3.750000
2,17.125041,14.795147,0,0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988,38,118.0,143.0,8.000000,4.861111
3,14.389425,12.216867,0,1,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718,25,92.0,103.0,6.305556,4.305556
4,9.579175,9.100648,0,1,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248,16,62.0,68.0,5.784722,2.944444
5,23.598392,18.449570,0,1,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633,49,160.0,190.0,9.722222,6.527778
6,30.079292,22.385369,0,0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984,57,194.0,216.0,16.013889,8.888889
7,26.472550,20.542158,0,1,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458,60,180.0,215.0,10.902778,7.486111
8,21.867940,17.016372,0,1,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488,43,142.0,163.0,8.527778,6.777778
9,3.932653,4.244375,1,0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29,4,22.0,21.0,4.222222,1.444444


In [9]:
## Change data types to float
calDFFloat = pd.DataFrame(calDF, dtype='float')

In [10]:
calDFFloat.shape

(747, 1613)

In [11]:
## Drop columns with more than 5% missing values.
calDF_test = calDFFloat[calDFFloat.columns[calDFFloat.isnull().mean() < .05]]
calDF_test.shape

(747, 1392)

In [12]:
## Check how many columns have 0 std.
std_vals = calDF_test.std()

In [13]:
count = 0
for i in std_vals:
    
    if i == 0:
        count += 1
print(count)

134


In [14]:
## Remove std = 0 columns
calDF_ST = calDF_test[calDF_test.columns[calDF_test.std() != 0]]

In [15]:
calDF_ST.shape

(747, 1258)

In [40]:
## removeNA columns and remove zero std
def removeNAColumns(df, percent = 0.05):
    df = df.apply(pd.to_numeric,errors='coerce')
    naCutOff = df.shape[1]*percent
    return(df[df.columns[(df.isna().sum() < naCutOff).values]])
def removezerostd(df):
    df = removeNAColumns(df)
    zero = df[df.columns[df.std()!=0]]
    return zero


In [41]:
removezerostd(calDF)

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.003721,11.805899,0,0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505,26,88.0,104.0,7.027778,3.777778
1,13.423098,11.209659,0,3,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515,24,90.0,106.0,4.305556,3.750000
2,17.125041,14.795147,0,0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988,38,118.0,143.0,8.000000,4.861111
3,14.389425,12.216867,0,1,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718,25,92.0,103.0,6.305556,4.305556
4,9.579175,9.100648,0,1,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248,16,62.0,68.0,5.784722,2.944444
5,23.598392,18.449570,0,1,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633,49,160.0,190.0,9.722222,6.527778
6,30.079292,22.385369,0,0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984,57,194.0,216.0,16.013889,8.888889
7,26.472550,20.542158,0,1,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458,60,180.0,215.0,10.902778,7.486111
8,21.867940,17.016372,0,1,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488,43,142.0,163.0,8.527778,6.777778
9,3.932653,4.244375,1,0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29,4,22.0,21.0,4.222222,1.444444


In [16]:
def retrieveBioassayInfo(cid):
    aid = []
    score = []
    gene = []
    act = []
    try:
        page = urlopen('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/'+cid +'/assaysummary/XML')
        soup = BeautifulSoup(page, "html.parser")
        colNames = soup.find("columns").find_all("column")
        aidIndex = -1
        activityIndex = -1
        for i, col in enumerate(colNames):
            if col.text == "AID":
                aidIndex = i
            elif col.text == "Activity Value [uM]":
                scoreIndex = i
            elif col.text == "Bioactivity Outcome":
                activityIndex = i
            elif col.text == "Target GeneID":
                geneIndex = i
        for row in soup.find_all("row"):
            values= row.find_all("cell")
            if values[geneIndex].text !=  '':
                score.append(values[scoreIndex].text)
                gene.append(values[geneIndex].text)
                aid.append(values[aidIndex].text)
                act.append(values[activityIndex].text)
    except Exception as e: 
        pass
    return(aid, act, gene, score)

In [17]:
def collectDrugActiveTar(cid):
    (aid, act, gene, score) = retrieveBioassayInfo(cid)
    activeIndices = [i for i, x in enumerate(act) if x == "Active"]
    cidList = [cid]*len(activeIndices)
    df = pd.DataFrame({'cid':cidList,
                   'aid':[x for i, x in enumerate(aid) if i in activeIndices],
                    'score':[x for i, x in enumerate(score) if i in activeIndices], 
                    'entrezID':[x for i, x in enumerate(gene) if i in activeIndices]})
    return(df)

In [18]:
def createFullTargetDF(drugData):
    """
    This method creates a full df with all pubchem targets
    :param: a drug data frame with one column listed with a drug's cid and a second with drug id
    :return: data frame with all bioassay target information
    """
    fullTargetDF = pd.DataFrame(columns=['cid','aid','score',
                                         'entrezID', "drug_id"])
    for i, row in tqdm(drugData.iterrows(), total=drugData.shape[0]):
        ##Collect drug targets 
        pubChemDF = collectDrugActiveTar(row["PubCHEM"])
        
        ##get drug id information
        nameDF = pd.DataFrame({"drug_id": [row["drug_id"]]*pubChemDF.shape[0]})
        
        ##combine all drug information
        drugDF = pd.concat([pubChemDF,nameDF],axis=1)
        
        fullTargetDF = pd.concat([fullTargetDF, drugDF])
    ##write out
    return(fullTargetDF)

In [19]:
# Get CID and drugname from combined DF
cid_drug = combinedDF[['PubChem Compound ID','Name']]

In [20]:
cid_drug = cid_drug.rename(columns={"PubChem Compound ID": "PubCHEM", "Name": "drug_id"})

In [23]:
cid_drug.shape

(720, 2)

In [22]:
cid_drug = cid_drug.dropna()

In [24]:
cid_drug = cid_drug.astype({"PubCHEM": int, "drug_id": object})

In [25]:
cid_drug.head()

,PubCHEM,drug_id
0,6035,Broxuridine
1,2435,Brimonidine
2,2333,Benzbromarone
3,16960,Dexbrompheniramine
4,2431,Bretylium


In [27]:
cid_drug = cid_drug.astype(str)

In [29]:
cid_drug.head()

,PubCHEM,drug_id
0,6035,Broxuridine
1,2435,Brimonidine
2,2333,Benzbromarone
3,16960,Dexbrompheniramine
4,2431,Bretylium


In [30]:
createFullTargetDF(cid_drug)

100%|████████████████████████████████████████████████████████████████████████████████| 720/720 [17:26<00:00,  1.14s/it]


,cid,aid,score,entrezID,drug_id
0,6035,1458,14.1254,6607,Broxuridine
1,6035,1458,10,6607,Broxuridine
2,6035,1471,,3064,Broxuridine
3,6035,210689,5.06,7083,Broxuridine
4,6035,210691,25,7083,Broxuridine
5,6035,408386,5,888740,Broxuridine
6,6035,624032,4.0533,24660,Broxuridine
7,6035,624044,7.9373,24660,Broxuridine
8,6035,651751,39.8107,84385,Broxuridine
9,6035,651758,53.0804,3576,Broxuridine


In [31]:
##Make a data frame with one column called "PubCHEM" and other named "drug_id"
cid = {'PubCHEM': [x for x in combinedDF['PubChem Compound ID']], 'drug_id': [y for y in combinedDF['Name']]}
drug = pd.DataFrame(data=cid)


In [32]:
drug = drug.dropna()

In [33]:
drug = drug.astype({"PubCHEM": int, "drug_id": object})

In [34]:
drug = drug.astype(str)

In [42]:
## testing 5 cid&drug_id to see if it works
test_result = createFullTargetDF(drug.head())

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.18it/s]


In [43]:
test_result

,cid,aid,score,entrezID,drug_id
0,6035,1458,14.1254,6607,Broxuridine
1,6035,1458,10,6607,Broxuridine
2,6035,1471,,3064,Broxuridine
3,6035,210689,5.06,7083,Broxuridine
4,6035,210691,25,7083,Broxuridine
5,6035,408386,5,888740,Broxuridine
6,6035,624032,4.0533,24660,Broxuridine
7,6035,624044,7.9373,24660,Broxuridine
8,6035,651751,39.8107,84385,Broxuridine
9,6035,651758,53.0804,3576,Broxuridine
